<a href="https://colab.research.google.com/github/pdoren/analisis-estadistico-con-Python-G105/blob/main/Desaf%C3%ADo_Inferencia_e_hip%C3%B3tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafío - Inferencia e hipótesis

**Nombre**: Pablo Saavedra <br />
**Curso**: Análisis estadístico con Python (G105)


---



## Carga de librerías y definición de funciones de apoyo

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import math

pd.options.display.float_format = "{:,.3f}".format   # Visualizar 2 decimales


## 1. Carga de Datos
Carga los datos y genera un dataFrame que excluya las columnas correspondientes a la abreviatura del país, código de llamada, capital, código de moneda, ciudad más grande, lenguaje oficial, latitud y longitud. Considerando estos datos, realiza una inspección inicial sobre ellos, considerando gráficos. Concluye.

---

Carga de los datos con Pandas:

In [59]:
df = pd.read_csv('/content/drive/MyDrive/earnings.csv')

# Se eliminan duplicados si existen
df = df.drop_duplicates()

# Tratar valores faltantes
df.replace({np.nan: None}) # Reemplazar los NaN para tratarlos como nulos
if (df.isnull().sum().sum()/len(df) > 0.4): # Ver Nota 1
  df = df.dropna()
else:
  for col in df.select_dtypes(include="number").columns:
    df[col] = df[col].fillna(df[col].median())

df.describe()

,height,weight,male,earn,earnk,education,mother_education,father_education,walk,exercise,smokenow,tense,angry,age
count,"1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000","1,816.000"
mean,66.569,156.211,0.372,"21,147.296",21.147,13.234,13.527,13.546,5.303,3.049,1.746,1.421,1.421,42.934
std,3.832,34.367,0.483,"22,531.765",22.532,2.556,3.002,2.985,2.601,2.316,0.436,2.158,2.158,17.162
min,57.000,80.000,0.000,0.000,0.000,2.000,3.000,3.000,1.000,1.000,1.000,0.000,0.000,18.000
25%,64.000,130.000,0.000,"6,000.000",6.000,12.000,12.000,12.000,3.000,1.000,1.000,0.000,0.000,29.000
50%,66.000,150.000,0.000,"16,000.000",16.000,12.000,13.000,13.000,6.000,2.000,2.000,0.000,0.000,39.000
75%,69.250,180.000,1.000,"27,000.000",27.000,15.000,15.000,15.000,8.000,5.000,2.000,2.000,2.000,56.000
max,82.000,342.000,1.000,"400,000.000",400.000,18.000,99.000,99.000,8.000,7.000,2.000,7.000,7.000,91.000


> **Nota 1**: Para los valores faltantes se tomará el porcentaje de nulos y NaN por columna y si es menor a 40% se inputará la media, de caso contrario se amputará (eliminar) las filas con valores nulos en las columnas procesadas. ([Link Justificación criterio 40%](https://pmc.ncbi.nlm.nih.gov/articles/PMC6293424/))


## 1. Test de Hipótesis
Considerando las variables **earn**, **height** y **age**, vamos a suponer que nuestro dataset fuera una población completa.

**a**. Realiza 12 pruebas (4 por cada variable) con muestras de tamaño n = 25, en cada caso, con niveles de significancia diferentes pero inferiores a 0,1, para confirmar o rechazar las siguientes hipótesis considerando las alternativas. Crea para ello una fórmula que reciba los parámetros adecuados y responda "con nivel de confianza del ...%, se acepta la hipótesis nula/alternativa".

| earn                        | height                      | age                         |
|----------------------------|-----------------------------|-----------------------------|
| H₀: x̄ = 20000 <br> H₁: x̄ ≠ 20000 | H₀: x̄ = 55 <br> H₁: x̄ ≠ 55       | H₀: x̄ = 44 <br> H₁: x̄ ≠ 44       |
| H₀: x̄ = 35000 <br> H₁: x̄ ≠ 35000 | H₀: x̄ = 68 <br> H₁: x̄ ≠ 68       | H₀: x̄ = 56 <br> H₁: x̄ ≠ 56       |
| H₀: x̄ = 25000 <br> H₁: x̄ > 25000 | H₀: x̄ = 63 <br> H₁: x̄ > 63       | H₀: x̄ = 51 <br> H₁: x̄ > 51       |
| H₀: x̄ = 32000 <br> H₁: x̄ < 32000 | H₀: x̄ = 70 <br> H₁: x̄ < 70       | H₀: x̄ = 62 <br> H₁: x̄ < 62       |


In [60]:
data = dict()
data['earn'] = df['earn']
data['height'] = df['height']
data['age'] = df['age']

medias = dict()
medias['earn'] = [20000, 35000, 25000, 32000]
medias['height'] = [55, 68, 63, 70]
medias['age'] = [44, 56, 51, 62]

alternatives = [None, None, 'greater', 'less']

texto_prueba = [['H₀: x̄ = ', 'H₁: x̄ ≠ '],['H₀: x̄ = ', 'H₁: x̄ ≠ '], ['H₀: x̄ = ', 'H₁: x̄ > '], ['H₀: x̄ = ', 'H₁: x̄ < ']]

lista_significancias = [0.1, 0.05, 0.01, 0.001]

largo_linea = 80

df_pruebas = pd.DataFrame()

# Formula para obener resultado de prueba
def formula(_p_value, _significancia):
  if p_value <= significancia:
    return f"Con nivel de confianza del {(1-significancia)*100}%, se acepta la hipótesis alternativa"
  else:
    return f"Con nivel de confianza del {(1-significancia)*100}%, se acepta la hipótesis nula" # no necesariamente


for variable in data:
  values = data[variable].sample(n=25) # muestra aleatoria de 25 elementos
  print(largo_linea*'*')
  print(f'* Prueba de Hipotesis {variable}:')
  print(largo_linea*'*')

  for significancia in lista_significancias:
    print(f'Nivel de significancia: {significancia:.3f}')
    print(largo_linea*'-')
    for index, mu in enumerate(medias[variable]):
      print('Prueba:')
      prueba = texto_prueba[index][0] + str(mu) + " | "+ texto_prueba[index][1] + str(mu)
      print(prueba)
      if (alternatives[index] == None):
        t_stat, p_value = st.ttest_1samp(values, mu)
      else:
        t_stat, p_value = st.ttest_1samp(values, mu, alternative=alternatives[index])
      print(f"Valor t: {t_stat:.3f}")
      print(f"Valor p: {p_value:.3f}")

      resultado = formula(p_value, significancia)
      # Mostrar resultado
      print(resultado)

      # Guardar datos
      data_prueba = dict()
      data_prueba['variable'] = [variable]
      data_prueba['prueba'] = [prueba]
      data_prueba['significancia'] = [significancia]
      data_prueba['valor t'] = [t_stat]
      data_prueba['valor p'] = [p_value]
      data_prueba['media muestral'] = [values.mean()]
      data_prueba['media poblacional'] = [data[variable].mean()]
      data_prueba['std muestral'] = [values.std()]
      data_prueba['std poblacional'] = [data[variable].std()]
      data_prueba['resultado'] = [resultado]
      # Crear un DataFrame a partir del diccionario
      new_df = pd.DataFrame(data_prueba)
      # Concatenar los DataFrames
      df_pruebas = pd.concat([df_pruebas, new_df], ignore_index=True)

      print(largo_linea*'*')
    print(largo_linea*'*')
  print('\n')

********************************************************************************
* Prueba de Hipotesis earn:
********************************************************************************
Nivel de significancia: 0.100
--------------------------------------------------------------------------------
Prueba:
H₀: x̄ = 20000 | H₁: x̄ ≠ 20000
Valor t: -0.453
Valor p: 0.655
Con nivel de confianza del 90.0%, se acepta la hipótesis nula
********************************************************************************
Prueba:
H₀: x̄ = 35000 | H₁: x̄ ≠ 35000
Valor t: -4.034
Valor p: 0.000
Con nivel de confianza del 90.0%, se acepta la hipótesis alternativa
********************************************************************************
Prueba:
H₀: x̄ = 25000 | H₁: x̄ > 25000
Valor t: -1.647
Valor p: 0.944
Con nivel de confianza del 90.0%, se acepta la hipótesis nula
********************************************************************************
Prueba:
H₀: x̄ = 32000 | H₁: x̄ < 32000
Valor t: -

**b**. Calcula la media poblacional para cada variable. ¿Son correctos los resultados obtenidos por tus pruebas?

In [61]:
df_variable = df_pruebas[df_pruebas['variable'] == 'earn']
media_variable = df_variable['media muestral'].mean()
media_poblacional = df_variable['media poblacional'].mean()
std_variable = df_variable['std muestral'].mean()
std_poblacional = df_variable['std poblacional'].mean()
print(f'Media muestral: {media_variable:.3f}')
print(f'Media poblacional: {media_poblacional:.3f}')
print(f'Desviación muestral: {std_variable:.3f}')
print(f'Desviación poblacional: {std_poblacional:.3f}')
df_variable = df_variable.drop (['media muestral', 'media poblacional', 'std muestral', 'std poblacional'], axis = 1)
df_variable

Media muestral: 18102.400
Media poblacional: 21147.296
Desviación muestral: 20943.139
Desviación poblacional: 22531.765


,variable,prueba,significancia,valor t,valor p,resultado
0,earn,H₀: x̄ = 20000 | H₁: x̄ ≠ 20000,0.100,-0.453,0.655,"Con nivel de confianza del 90.0%, se acepta la..."
1,earn,H₀: x̄ = 35000 | H₁: x̄ ≠ 35000,0.100,-4.034,0.000,"Con nivel de confianza del 90.0%, se acepta la..."
2,earn,H₀: x̄ = 25000 | H₁: x̄ > 25000,0.100,-1.647,0.944,"Con nivel de confianza del 90.0%, se acepta la..."
3,earn,H₀: x̄ = 32000 | H₁: x̄ < 32000,0.100,-3.318,0.001,"Con nivel de confianza del 90.0%, se acepta la..."
4,earn,H₀: x̄ = 20000 | H₁: x̄ ≠ 20000,0.050,-0.453,0.655,"Con nivel de confianza del 95.0%, se acepta la..."
5,earn,H₀: x̄ = 35000 | H₁: x̄ ≠ 35000,0.050,-4.034,0.000,"Con nivel de confianza del 95.0%, se acepta la..."
6,earn,H₀: x̄ = 25000 | H₁: x̄ > 25000,0.050,-1.647,0.944,"Con nivel de confianza del 95.0%, se acepta la..."
7,earn,H₀: x̄ = 32000 | H₁: x̄ < 32000,0.050,-3.318,0.001,"Con nivel de confianza del 95.0%, se acepta la..."
8,earn,H₀: x̄ = 20000 | H₁: x̄ ≠ 20000,0.010,-0.453,0.655,"Con nivel de confianza del 99.0%, se acepta la..."
9,earn,H₀: x̄ = 35000 | H₁: x̄ ≠ 35000,0.010,-4.034,0.000,"Con nivel de confianza del 99.0%, se acepta la..."


**Resultado**:

- H₀: x̄ = 20000 | H₁: x̄ ≠ 20000: El valor de p es pequeño lo que indica que no se rechaza H₀, pero también lo es el valor de t (cercano a 0) lo que indica cierta aleatoriedad en la muestra y por tanto no hay evidencia significativa
- H₀: x̄ = 35000 | H₁: x̄ ≠ 35000: El valor de p es muy bajo y el valor de t mayor que 0, esto indica que se acepta la hipotesis alternativa debido a que hay evidencia significativa (x̄ ≠ 35000 ≠ 21147.296).
- H₀: x̄ = 25000 | H₁: x̄ > 25000: El valor de p es grande cercano a 1 y el valor de t mayor a 1.5 (valor absoluto), esto indica que aunque no se rechace la hipótesis nula es posible que exista una diferencia. Resultado final no concluyente.
- H₀: x̄ = 32000 | H₁: x̄ < 32000: El valor de p es muy bajo y el valor de t mayor que 0, esto indica que se acepta la hipotesis alternativa debido a que hay evidencia significativa (x̄ < 32000 > 21147.296).

In [62]:
df_variable = df_pruebas[df_pruebas['variable'] == 'height']
media_variable = df_variable['media muestral'].mean()
media_poblacional = df_variable['media poblacional'].mean()
std_variable = df_variable['std muestral'].mean()
std_poblacional = df_variable['std poblacional'].mean()
print(f'Media muestral: {media_variable:.3f}')
print(f'Media poblacional: {media_poblacional:.3f}')
print(f'Desviación muestral: {std_variable:.3f}')
print(f'Desviación poblacional: {std_poblacional:.3f}')
df_variable = df_variable.drop (['media muestral', 'media poblacional', 'std muestral', 'std poblacional'], axis = 1)
df_variable

Media muestral: 66.280
Media poblacional: 66.569
Desviación muestral: 4.159
Desviación poblacional: 3.832


,variable,prueba,significancia,valor t,valor p,resultado
16,height,H₀: x̄ = 55 | H₁: x̄ ≠ 55,0.100,13.562,0.000,"Con nivel de confianza del 90.0%, se acepta la..."
17,height,H₀: x̄ = 68 | H₁: x̄ ≠ 68,0.100,-2.068,0.050,"Con nivel de confianza del 90.0%, se acepta la..."
18,height,H₀: x̄ = 63 | H₁: x̄ > 63,0.100,3.944,0.000,"Con nivel de confianza del 90.0%, se acepta la..."
19,height,H₀: x̄ = 70 | H₁: x̄ < 70,0.100,-4.473,0.000,"Con nivel de confianza del 90.0%, se acepta la..."
20,height,H₀: x̄ = 55 | H₁: x̄ ≠ 55,0.050,13.562,0.000,"Con nivel de confianza del 95.0%, se acepta la..."
21,height,H₀: x̄ = 68 | H₁: x̄ ≠ 68,0.050,-2.068,0.050,"Con nivel de confianza del 95.0%, se acepta la..."
22,height,H₀: x̄ = 63 | H₁: x̄ > 63,0.050,3.944,0.000,"Con nivel de confianza del 95.0%, se acepta la..."
23,height,H₀: x̄ = 70 | H₁: x̄ < 70,0.050,-4.473,0.000,"Con nivel de confianza del 95.0%, se acepta la..."
24,height,H₀: x̄ = 55 | H₁: x̄ ≠ 55,0.010,13.562,0.000,"Con nivel de confianza del 99.0%, se acepta la..."
25,height,H₀: x̄ = 68 | H₁: x̄ ≠ 68,0.010,-2.068,0.050,"Con nivel de confianza del 99.0%, se acepta la..."


**Resultado**:

- H₀: x̄ = 55 | H₁: x̄ ≠ 55: El valor de p es muy pequeño lo que indica que se rechaza H₀ y se acepta la hipótesis alternativa, además el valor de t es muy grande lo que indica alta significancia estadística (x̄ ≠ 55 ≠ 66.569)  
- H₀: x̄ = 68 | H₁: x̄ ≠ 68: El valor de p es bajo y el valor de t mayor que 1.5, esto indica que se acepta la hipotesis alternativa debido a que hay evidencia significativa (x̄ ≠ 68 ≠ 66.569).
- H₀: x̄ = 63 | H₁: x̄ > 63: El valor de p es bajo y el valor de t mayor que 1.5, esto indica que se acepta la hipotesis alternativa debido a que hay evidencia significativa (x̄ > 63 < 66.569).
- H₀: x̄ = 70 | H₁: x̄ < 70: El valor de p es muy bajo y el valor de t mayor que 1.5, esto indica que se acepta la hipotesis alternativa debido a que hay evidencia significativa (x̄ < 70 > 66.569).

In [63]:
df_variable = df_pruebas[df_pruebas['variable'] == 'age']
media_variable = df_variable['media muestral'].mean()
media_poblacional = df_variable['media poblacional'].mean()
std_variable = df_variable['std muestral'].mean()
std_poblacional = df_variable['std poblacional'].mean()
print(f'Media muestral: {media_variable:.3f}')
print(f'Media poblacional: {media_poblacional:.3f}')
print(f'Desviación muestral: {std_variable:.3f}')
print(f'Desviación poblacional: {std_poblacional:.3f}')
df_variable = df_variable.drop (['media muestral', 'media poblacional', 'std muestral', 'std poblacional'], axis = 1)
df_variable

Media muestral: 42.480
Media poblacional: 42.934
Desviación muestral: 16.055
Desviación poblacional: 17.162


,variable,prueba,significancia,valor t,valor p,resultado
32,age,H₀: x̄ = 44 | H₁: x̄ ≠ 44,0.100,-0.473,0.640,"Con nivel de confianza del 90.0%, se acepta la..."
33,age,H₀: x̄ = 56 | H₁: x̄ ≠ 56,0.100,-4.211,0.000,"Con nivel de confianza del 90.0%, se acepta la..."
34,age,H₀: x̄ = 51 | H₁: x̄ > 51,0.100,-2.653,0.993,"Con nivel de confianza del 90.0%, se acepta la..."
35,age,H₀: x̄ = 62 | H₁: x̄ < 62,0.100,-6.079,0.000,"Con nivel de confianza del 90.0%, se acepta la..."
36,age,H₀: x̄ = 44 | H₁: x̄ ≠ 44,0.050,-0.473,0.640,"Con nivel de confianza del 95.0%, se acepta la..."
37,age,H₀: x̄ = 56 | H₁: x̄ ≠ 56,0.050,-4.211,0.000,"Con nivel de confianza del 95.0%, se acepta la..."
38,age,H₀: x̄ = 51 | H₁: x̄ > 51,0.050,-2.653,0.993,"Con nivel de confianza del 95.0%, se acepta la..."
39,age,H₀: x̄ = 62 | H₁: x̄ < 62,0.050,-6.079,0.000,"Con nivel de confianza del 95.0%, se acepta la..."
40,age,H₀: x̄ = 44 | H₁: x̄ ≠ 44,0.010,-0.473,0.640,"Con nivel de confianza del 99.0%, se acepta la..."
41,age,H₀: x̄ = 56 | H₁: x̄ ≠ 56,0.010,-4.211,0.000,"Con nivel de confianza del 99.0%, se acepta la..."


**Resultado**:

- H₀: x̄ = 44 | H₁: x̄ ≠ 44: El valor de p es grande lo que indica que no se rechaza H₀, pero el valor de t es muy pequeño lo que indica que no hay evidencia significativa (H₀ está cercano a le media muestral)
- H₀: x̄ = 56 | H₁: x̄ ≠ 56: El valor de p es bajo y el valor de t mayor que 1.5, esto indica que se acepta la hipotesis alternativa debido a que hay evidencia significativa (x̄ ≠ 56 ≠ 42.934).
- H₀: x̄ = 51 | H₁: x̄ > 51: El valor de p es alto y el valor de t mayor que 1.5, esto indica que se acepta la hipotesis nula, pero posible diferencia no concluyente.
- H₀: x̄ = 62 | H₁: x̄ < 62: El valor de p es muy bajo y el valor de t mayor que 1.5, esto indica que se acepta la hipotesis alternativa debido a que hay evidencia significativa (x̄ < 62 > 42.934).

## 2. Función
Crea una función que estime la proporción de hombres en el dataset, considerando una muestra de 50 individuos, con niveles de significancia de 0,05 y 0,01. Escribe con palabras tus resultados ejecutar la función e interpreta. Compara con la proporción real.

In [214]:
import statsmodels.api as sm
from scipy.stats import binomtest



def get_media_hombres(_data, _significancia):
  k = _data[_data['male'] == 1]['male'].count()
  n = len(_data)
  intervalo_confianza = sm.stats.proportion_confint(k, n, alpha=_significancia)
  a = intervalo_confianza[0] # Límite Inferior Intervalo
  b = intervalo_confianza[1] # Límite Superior Intervalo
  p = a # p: proporción de hombres. Se inicia con límite inferior del intervalo de confianza
  d = (b - a) * 0.01 # paso para las pruebas
  while p > b:
    result = binomtest(k=k, n=n, p=p) # Prueba de hipótesis
    if (result.pvalue <= _significancia):
      p += d # Se suma el paso para la próxima prueba
    else:
      break
  return p


media_poblacional = df['male'].mean() # Media poblacional
print(f'Proporción real de hombres: {media_poblacional:.3f}')
sample = df.sample(n=50, random_state=42) # Muestra de 50 individuos

# Estimación de media poblacional según muestra y significancia
media_estimada_1 = get_media_hombres(sample, 0.05)
media_estimada_2 = get_media_hombres(sample, 0.01)
print(f'Proporción estimada de hombres con nivel de significancia 0.05: {media_estimada_1:.3f}')
print(f'Proporción estimada de hombres con nivel de significancia 0.01: {media_estimada_2:.3f}')


Proporción real de hombres: 0.372
Proporción estimada de hombres con nivel de significancia 0.05: 0.402
Proporción estimada de hombres con nivel de significancia 0.01: 0.358


> **Función get_media_hombres**: función para media poblacional a partir de una muestra. Para obtener la estimación primero se calcula un intervalo de confianza y a partir de ese intervalo se comienza a iterar de forma ordenada sobre los valores en el intervalo calculando su valor p y validando si este es mayor a la significancia entregada como parámetro.

La proporcional real de hombres es de 0.372 y la estimación obtenida fue de 0.402 y 0.358 para los niveles de significación 0.05 y 0.01 respectivamente. El error fue de -0.030 y +0.014 para los niveles de significación 0.05 y 0.01 respectivamente. El error bajó al reducir el nivel de significación.

### 3. Influencia de genero en sueldo

Considerando el dataset como una muestra, verifica si el género (male=1 significa ‘hombre”) influye sobre el sueldo “earn” de las personas. Explica y justifica tu procedimiento.



Para revisar la influencia sobre el sueldo entre las personas se realizará una prueba de hipótesis para muestras independientes. Ya que vamos a comparar las medias de dos grupos distintos se utilizará t-test de dos muestras independientes.

#### Formulación de hipótesis:

**Hipótesis nula (H₀):** No hay diferencia significativa en el sueldo entre hombres y mujeres.

**Hipótesis alternativa (H₁):** Hay una diferencia significativa en el sueldo entre hombres y mujeres.

In [220]:
hombre = df[df['male'] == 1]['earn']
mujer = df[df['male'] == 0]['earn']


# Realizar una prueba de hipótesis (prueba t de muestras independientes)
t_stat, p_value = st.ttest_ind(hombre, mujer, equal_var=False)

# Nivel de significancia (alfa)
alpha = 0.05

# Comprobar si el valor p es menor que alfa para rechazar la hipótesis nula
if p_value < alpha:
    print("Resultado: Se rechaza la hipótesis nula")
else:
    print("Resultado: No se rechaza la hipótesis nula")

# Imprimir el valor t y el valor p
print(f"Valor t: {t_stat}")
print(f"Valor p: {p_value}")


Resultado: Se rechaza la hipótesis nula
Valor t: 11.86459381795286
Valor p: 2.7373648004575493e-30


### Resultados:

Debido a que el valor p << 0.05 (y valor de t >> 2.5) hay evidencia suficiente para rechazar la hipótesis nula y concluir que existe una diferencia significativa entre el sueldo de hombres y mujeres. Por lo tanto, influye el genero en el sueldo.

In [221]:
media_hombre = hombre.mean()
media_mujer = mujer.mean()
print(f'Media hombre: {media_hombre:.3f}')
print(f'Media mujer: {media_mujer:.3f}')

Media hombre: 30105.179
Media mujer: 15847.935


El sueldo del hombre en promedio es el doble que el de la mujer.